In [35]:
# Mapping card denomation to number
value_dict = {'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
value_dict.update((str(x), x) for x in range(2,10))

import enum


class Result(enum.Enum):
    #Enumeration for result: WIN or LOSS
    WIN = 1
    LOSS = 0

class PokerHand:
  #Class for creating poker hands and comparing them
  #Example of raw card as input: "TC TH 5C 5H KH"
  #A space will used as a separator for each card
  def __init__(self, strHand):
    self.hand = strHand
  
  def obtain_value(self,hand):
    #Sort the hand based on its card value. 
    values = sorted([c[0] for c in hand], reverse=True)
    #Obtaining all suits for the hand
    suits = [c[1] for c in hand]
    
    #It is straight if all items are sorted and in ascending order
    straight = (values == list(range(values[0], values[0]-5, -1)))
    #It is a flush if all suits are equal
    flush = all(s == suits[0] for s in suits)

    if straight and flush:
        #If it is order and the first item is 10 then it is royal straight flush
        if values[0] == 10:
            return 9, None
        #Otherwise it is only straight flush
        else: return 8, max(values)

    pairs = []
    pair_present = False
    three_of_a_kind = False
    three_value = None
    #Counting unique values for each hand
    for v in set(values):
        if values.count(v) == 4:
            #If contains 4 unique values then it is four of a kind
            #Return ranking and highest value
            return 7, v
        if values.count(v) == 3:
            three_of_a_kind = True
            three_value = v
        if values.count(v) == 2:
            pair_present = True
            pairs.append(v)
    
    #pairs[0]: highest value in case of pair detected
    #three_value: highest value if its three of a kind, otherwise it'd be four of a kind or something else
    #pairs: If two pair, compare both cards value pairs
    #max(values): since it is straight, get the maximum value for the different suits
    
    if three_of_a_kind and pair_present: return 6, (three_value, pairs[0])
    if flush: return 5, None
    if straight: return 4, max(values) 
    if three_of_a_kind: return 3, three_value
    if len(pairs) == 2: return 2, pairs
    if len(pairs) == 1: return 1, pairs[0]
    return 0, max(values)

  def verify_tie(self,hand1, hand2,hand1_info,hand2_info):
    #Compare the information collected
    
    if hand1_info != hand2_info:
        return (hand1_info > hand2_info)
    
    values1 = sorted((c[0] for c in hand1), reverse=True)
    values2 = sorted((c[0] for c in hand2), reverse=True)
    
    return (values1 > values2)

  def compare_with(self,pokerHand):
    #Split the string into cards
    hand1 = self.hand.split(' ')
    
    tuplaHand1 = []
    #We map the string cards to its corresponding numeric value
    for card in hand1:
        try:
            value = int(card[0])
        except:
            value = value_dict[card[0]]
        #Append the numeric value and its suit to a tuple for comparing
        tuplaHand1.append((value, card[1]))
    #Same process for the second hand
    hand2 = pokerHand.hand.split(' ')
    
    tuplaHand2 = []
    for card in hand2:
        try:
            value = int(card[0])
        except:
            value = value_dict[card[0]]
        tuplaHand2.append((value, card[1]))
    
    #Obtain ranking to the hand, and additional information in case of a tie
    rank1,info1 = self.obtain_value(tuplaHand1)
    rank2,info2 = self.obtain_value(tuplaHand2)
    
    if rank1 > rank2:
        return Result.WIN
    elif rank1 == rank2 and self.verify_tie(tuplaHand1, tuplaHand2, info1, info2):
        return Result.WIN
    else:
        return Result.LOSS


In [36]:
import unittest
class PokerHandTestCases(unittest.TestCase):
    def test_pokerhand(self):
        self.assertTrue(PokerHand("TC TH 5C 5H KH").compare_with(PokerHand("9C 9H 5C 5H AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS TD KC JC 7C").compare_with(PokerHand("JS JC AS KC TD")) == Result.LOSS)
        self.assertTrue(PokerHand("7H 7C QC JS TS").compare_with(PokerHand("7D 7C JS TS 6D")) == Result.WIN)
        self.assertTrue(PokerHand("5S 5D 8C 7S 6H").compare_with(PokerHand("7D 7S 5S 5D JS")) == Result.LOSS)
        self.assertTrue(PokerHand("AS AD KD 7C 3D").compare_with(PokerHand("AD AH KD 7C 4S")) == Result.LOSS)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("AC AH AS AS KS")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("TC JS QC KS AC").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JC JS JD TH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("TS TH TD JH JD").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)

In [37]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.019s

OK
